In [ ]:
import open3d as o3d

# source = o3d.io.read_point_cloud("source.e57")
# target = o3d.io.read_point_cloud("target.pcd")

In [10]:
def sample_test(sou, **kwargs):
    print(kwargs.keys())
    print(kwargs['voxel_size'])
    print(kwargs['distance_threshold'])

In [11]:
sample_test(11, voxel_size=10, distance_threshold=20)

dict_keys(['voxel_size', 'distance_threshold'])
10
20


In [ ]:
def merge_e57_point_clouds(source_path, target_path, output_path, **kwargs):
    """
    Merge two overlapping .e57 point clouds into a single point cloud.

    Args:
        source_path (str): Path to the source .e57 file.
        target_path (str): Path to the target .e57 file.
        output_path (str): Path to save the merged point cloud as .e57 or other formats.

    Returns:
        merged_pcd (open3d.geometry.PointCloud): Merged point cloud.
    """
    # Load source and target .e57 files
    print("Loading point clouds...")
    source_pcd = o3d.io.read_point_cloud(source_path)
    target_pcd = o3d.io.read_point_cloud(target_path)

    # Downsample point clouds for faster processing
    print("Downsampling point clouds...")
    if kwargs['voxel_size'] is not None:
        voxel_size = kwargs['voxel_size']
    else:
        voxel_size = 0.05  # Adjust voxel size as needed

    if kwargs['distance_threshold'] is not None:
        distance_threshold = kwargs['distance_threshold']
    else:
        distance_threshold = voxel_size * 2  # Adjust threshold as needed

    source_down = source_pcd.voxel_down_sample(voxel_size)
    target_down = target_pcd.voxel_down_sample(voxel_size)

    # Compute FPFH feature descriptors for both point clouds
    print("Computing FPFH features...")
    source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        source_down, o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5, max_nn=100)
    )
    target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        target_down, o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5, max_nn=100)
    )

    # Perform RANSAC-based initial alignment
    print("Performing initial alignment with RANSAC...")
    
    result_ransac = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh,
        max_correspondence_distance=distance_threshold,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        ransac_n=4,
        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(400000, 500)
    )

    print("Initial alignment result:")
    print(result_ransac)

    # Perform fine-tuning with ICP
    print("Refining alignment with ICP...")
    result_icp = o3d.pipelines.registration.registration_icp(
        source_pcd, target_pcd, max_correspondence_distance=voxel_size,
        init=result_ransac.transformation,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint()
    )

    print("ICP alignment result:")
    print(result_icp)

    # Apply transformation to source point cloud
    print("Applying transformation to source point cloud...")
    source_pcd.transform(result_icp.transformation)

    # Merge the point clouds
    print("Merging point clouds...")
    merged_pcd = source_pcd + target_pcd

    # Save the merged point cloud
    print(f"Saving merged point cloud to {output_path}...")
    o3d.io.write_point_cloud(output_path, merged_pcd)

    print("Point clouds merged successfully.")
    return merged_pcd

voxel_size:다운샘플링 크기와 정렬의 정밀도에 영향을 미칩니다. 겹치는 영역이 작거나 더 세밀한 정렬이 필요할 경우 값을 줄이세요.

distance_threshold:RANSAC 및 ICP 정렬 시의 대응점 거리 허용값. 겹치는 영역이 적다면 값을 늘려야 합니다.

max_correspondence_distance: ICP의 대응점 거리 허용값으로, 병합 정밀도에 영향을 줍니다.

In [ ]:
# Example usage
source_path = "source.e57"
target_path = "target.e57"
output_path = "merged.e57"

merged_pcd = merge_e57_point_clouds(source_path, target_path, output_path)

# # If overlapped points erased,
# merged_pcd = merged_pcd.voxel_down_sample(voxel_size=0.01)

# # Statistical noise removal
# cl, ind = merged_pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
# merged_pcd = merged_pcd.select_by_index(ind)

# Visualize the merged point cloud
o3d.visualization.draw_geometries([merged_pcd])